# MIMIC-III M3Care Baseline

### Load modules and define constants.

In [1]:
from dataset import MIMICDataset
from m3care import M3Care
from comp import MLP
from synth import ( NLEmbedder, TimeSeriesNLEmbedder )
from torch import nn
from vocab import Vocab
from torch.nn import functional as F
import torch
import os

In [2]:
processed_dir = './data/processed'

### Load Datasets

In [3]:
train_ds = MIMICDataset(processed_dir, True)
# test_ds = MIMICDataset(processed_dir, False)

In [4]:
vocab = Vocab.from_json(os.path.join(processed_dir, 'vocab.json'))

In [5]:
DEM_DIM = 18
VIT_DIM = 105
ITV_DIM = 14

### Instantiate Unimodal Extraction Models

In [6]:
EMB_DIM = 512

DROPOUT = 0.3

In [8]:
dem_mdl = MLP(in_dim=DEM_DIM, hidden_dim=[128,192,256], out_dim=EMB_DIM, bias=True, relu=True, norm=True)

In [9]:
vit_mdl = nn.GRU(VIT_DIM, EMB_DIM*2, 2, batch_first=True, dropout=DROPOUT)

In [10]:
itv_mdl = nn.GRU(ITV_DIM, EMB_DIM*2, 2, batch_first=True, dropout=DROPOUT)

In [11]:
nst_mdl = NLEmbedder(vocab, 16, EMB_DIM, 8, 2048, dropout=DROPOUT)

In [12]:
nts_mdl = TimeSeriesNLEmbedder(vocab, 16, EMB_DIM, 8, 2048, dropout=DROPOUT)

### Instantiate M3Care Model

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [43]:
model = M3Care(dem_mdl, vit_mdl, itv_mdl, nst_mdl, nts_mdl, EMB_DIM, 2, device, 0.7).to(device)

In [44]:
sample = train_ds[:32]

In [45]:
s_dem = torch.from_numpy(sample[0]).float().to(device)
s_vit = torch.from_numpy(sample[1]).float().to(device)
s_itv = torch.from_numpy(sample[2]).float().to(device)
s_nst = torch.from_numpy(sample[3]).float().to(device)
s_nts = sample[4]
s_miss = sample[5]

In [46]:
model(s_dem, s_vit, s_itv, s_nst, s_nts, s_miss)

RuntimeError: expand(torch.cuda.FloatTensor{[41, 521]}, size=[512]): the number of sizes provided (1) must be greater or equal to the number of dimensions in the tensor (2)